In [2]:
%matplotlib inline

import os
import sys
import copy
import numpy as np
#np.seterr(over='ignore')
import scipy as sp

# filtering Astropy warnings
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

# autoreload user modules functionality
%load_ext autoreload
%autoreload 2

sys.path.append('../lib/')
from hdmclouds_v1 import *
from utils import *
from graph import *
from preprocessing import *
from gmr import *

from ipywidgets import interact, interactive, fixed, FloatSlider, IntSlider
import aplpy
import matplotlib
import matplotlib.pyplot as plt; plt.show()
matplotlib.rcParams.update({'font.size': 13})

from pycupid import *

save_path = "/Users/martin/Dropbox/Aplicaciones/ShareLaTeX/HDMC-v1/figures/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
fits_path = '../data/images/Orion.cont.image.fits'
data, wcs, hdu = load_data(fits_path)
base_level = estimate_rms(data)

In [3]:
hdmc = HDMClouds(data, back_level=0.089, wcs=wcs, verbose=False, n_center=200)

/Users/martin/anaconda3/lib/python3.6/site-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
../lib/points_generation.py:202: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum()
../lib/points_generation.py:191: RuntimeWarning: invalid value encountered in less
  sel = np.random.choice(points_indexes, size=1 , p=prob.ravel(), replace=False)[0]
../lib/points_generation.py:191: RuntimeWarning: invalid value encountered in greater
  sel = np.random.choice(points_indexes, size=1 , p=prob.ravel(), replace=False)[0]


No more points can be selected: Only 35 point from the 40 were selected.


### Gaussian mixture approximation profiling

In [4]:
%%prun -s tottime -q -l 30 -T gma_prof
hdmc.build_gmr()

NameError: name 'prune' is not defined

* The __total time__ (`tottime`) represents how long the interpreter stays in a given function, excluding the time spent in calls to subfunctions.
* The __cumulative time__ (`cumtime`) is similar but includes the time spent in calls to subfunctions.

In [ ]:
print(open('gma_prof', 'r').read())

### Gaussian mixture reduction profiling

In [20]:
%%prun -s tottime -q -l 30 -T gmr_prof
xc, yc, c, sig = hdmc.get_params_mapped()
mu = np.vstack([xc,yc]).T
sd, htree = gaussian_reduction(c, mu, sig, 1, verbose=False)

 
*** Profile printout saved to text file 'gmr_prof'. 


In [21]:
print(open('gmr_prof', 'r').read())

         440901 function calls in 2.209 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.107    1.107    2.209    2.209 gmr.py:203(gaussian_reduction)
   437989    1.098    0.000    1.098    0.000 gmr.py:189(kl_diss)
      137    0.001    0.000    0.001    0.000 gmr.py:71(merge)
        1    0.001    0.001    0.002    0.002 gmr.py:218(<listcomp>)
      138    0.000    0.000    0.001    0.000 twodim_base.py:140(eye)
      411    0.000    0.000    0.000    0.000 {built-in method builtins.max}
        1    0.000    0.000    2.209    2.209 <string>:2(<module>)
      138    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.zeros}
      138    0.000    0.000    0.001    0.000 numeric.py:2157(identity)
      137    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
      138    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
      411    0.000

## Profiling after improvements

In [2]:
from hdmclouds_v1 import *


In [3]:
fits_path = '../data/images/Orion.cont.image.fits'
data, wcs, hdu = load_data(fits_path)
base_level = estimate_rms(data)

In [4]:
hdmc = HDMClouds(data, back_level=0.089, wcs=wcs, verbose=False, n_center=200)

/Users/martin/anaconda3/lib/python3.6/site-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


In [5]:
%%prun -s tottime -q -l 30 -T gma_prof2
hdmc.build_gmr()

xtol: 1e-17
ftol: 1e-17
 
*** Profile printout saved to text file 'gma_prof2'. 


In [6]:
print(open('gma_prof2', 'r').read())

         1169523 function calls (1167311 primitive calls) in 66.459 seconds

   Ordered by: internal time
   List reduced from 209 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    16386   43.311    0.003   43.311    0.003 {fgm_eval.gm_eval_trunc_thread}
       11   18.471    1.679   66.216    6.020 {built-in method scipy.optimize._minpack._lmdif}
    49158    1.190    0.000    2.356    0.000 necompiler.py:750(evaluate)
    16386    0.717    0.000    1.425    0.000 hdmclouds_v1.py:36(d1psi1)
    16386    0.470    0.000    0.470    0.000 utils.py:136(sig_mapping)
    16386    0.469    0.000   47.792    0.003 hdmclouds_v1.py:346(F)
    16386    0.332    0.000    0.332    0.000 {fgm_eval.gm_eval}
    49158    0.298    0.000    0.493    0.000 necompiler.py:725(getArguments)
    49161    0.268    0.000    0.354    0.000 necompiler.py:534(getContext)
    98386    0.104    0.000    0.104    0.000 {built-in method numpy.core.multiarray.a